In [1]:
#Packages
import pandas as pd
from pandas_datareader import data
import datetime as dt
from datetime import timedelta, date
import matplotlib.pyplot as plt
from scipy import stats
import os

#Variables
dateRangeVar = 180
slopeThresh = .1
varThresh = .05
peakThresh = 10
troughThresh = 10
buyAmt = 100
sellAmt = 100
newOrOld = 'new'
startingMoney = 10000

#(yes/no) Calculate Slope and Var thresh?
slopeVarCalc = 'yes'

#make sure to enter your path
excelOutputFolder = os.path.dirname(os.path.abspath('Stock Screen Algo Notebook'))+'\\'

#Create Clean Reset DF
statsDFBlank = pd.DataFrame(columns = ['Symbol', 'LastPrice', 'Days', 'StDev', 'Avg', 'Slope', 'Std/Avg', '#ofPeaks', '#ofTroughs'])
statsDF = statsDFBlank

#set date range
endDate = date.today()
dateRange = dt.timedelta(dateRangeVar)
startDate = endDate - dateRange

#Pull all S&P stocks
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
w = wiki[0]
w = w['Symbol']
w = w.reset_index()
iterations = len(w)

#Generate Export DF
for x in range(iterations): 
    try:
        #Pick a ticker
        myTicker = w['Symbol'].iloc[x]

        #List Prices
        Prices = data.DataReader([myTicker], 'yahoo', start=startDate, end=endDate)         
        Prices = Prices['Adj Close']
        Prices = Prices.reset_index()

        #Find Stats
        stdPrice = Prices.std()[myTicker]
        avgPrice = Prices.mean()[myTicker]
        tradeDays = Prices.count()[myTicker]
        
        #Total Peaks and Troughs
        Prices['Peak'] = Prices[myTicker].apply(lambda x: 1 if x >= (avgPrice + stdPrice)  else 0)
        Prices['Trough'] = Prices[myTicker].apply(lambda x: 1 if x <= (avgPrice - stdPrice)  else 0)
        
        #Assign Peaks and Troughs
        totalPeaks = Prices.sum()['Peak']
        totalTroughs = Prices.sum()['Trough']

        #Find slope
        slope, intercept, r, p, se = stats.linregress(Prices.index, Prices[myTicker])

        #Select most recent price
        mostrecent = Prices[myTicker].iloc[len(Prices)-1]

        #How volitaile
        percentStd = stdPrice / avgPrice

        #Add Export Data
        exData = {'Symbol': myTicker,
            'LastPrice': mostrecent,
            'Days': tradeDays,
            'StDev': stdPrice,
            'Avg': avgPrice,
            'Slope': slope,
            'Std/Avg': percentStd,
            '#ofPeaks': totalPeaks,
            '#ofTroughs': totalTroughs}
        statsDF = statsDF.append(exData, ignore_index = True)
    except:
        continue
        
#generate describe df
statsStatsDF = statsDF.describe()

#functions to set auto-limits
if slopeVarCalc == 'yes':
    varThresh = statsStatsDF['Std/Avg'].iloc[4]
    slopeThresh = statsStatsDF['Slope'].iloc[5]
    peakThresh = statsStatsDF['#ofPeaks'].iloc[5]
    troughThresh = statsStatsDF['#ofTroughs'].iloc[5]
    
#apply Buy and Sell Price and Logic
statsDFTwo = statsDF
statsDFTwo['buyPrice'] = statsDFTwo['Avg'] - statsDFTwo['StDev']
statsDFTwo['sellPrice'] = statsDFTwo['Avg'] + statsDFTwo['StDev']
statsDFTwo.loc[statsDFTwo['LastPrice'] <= statsDFTwo['buyPrice'], 'Buy?'] = 'yes' 
statsDFTwo.loc[statsDFTwo['LastPrice'] >= statsDFTwo['sellPrice'], 'Sell?'] = 'yes'

statsDFTwo=statsDFTwo.fillna('no')

def f(row):
    if row['Buy?'] == 'yes' or row['Sell?'] == 'yes':
        val = 'yes'
    else:
        val = 'no'
    return val

statsDFTwo['Buy or Sell?'] = statsDFTwo.apply(f, axis=1)

def s(row):
    if abs(row['Slope']) <= slopeThresh and row['Std/Avg'] >= varThresh and row['#ofPeaks'] >= peakThresh and row['#ofTroughs'] >= troughThresh:
        val = 'yes'
    else:
        val = 'no'
    return val
statsDFTwo['Qualifying Stock?'] = statsDFTwo.apply(s, axis=1)

#filter  for only reccomended buys and sells
statsDFThree = statsDFTwo[statsDFTwo['Qualifying Stock?'] == 'yes']
        
#Export all data to excel
#statsDF.to_excel(excelOutputFolder+"StockStatsExport.xlsx")
statsDFTwo.to_excel(excelOutputFolder+"StockStatsExportwithCalcs.xlsx")
statsDFThree.to_excel(excelOutputFolder+"StockStatsExportwithCalcsRefined.xlsx")   

#------BEGIN CODE FOR TRADING-----

#Function to reset Owned Stocks
if newOrOld == 'new':
    stockLedger = pd.DataFrame(columns = ['Date', 'Buy/Sell', 'Symbol', 'Price', 'Shares', 'Amount', 'cashBalance'])
    startingBalance = startingMoney
    newBal = startingBalance
elif newOrOld == 'old':
    stockLedger = pd.read_excel(excelOutputFolder+"Stock Ledger.xlsx", index_col=0)
    aggHoldings = pd.read_excel(excelOutputFolder+"Current Holdings.xlsx", index_col=0)
    newBal = aggHoldings[aggHoldings['Symbol'] == 'Cash']['value']
    newBal = int(newBal)        

#Function to place a trade
def placeTrade(action, amount, ticker):
    global newBal
    global stockLedger
    
    sharePrice = statsDFTwo[statsDFTwo['Symbol']==ticker]['LastPrice']
    sharePrice = int(sharePrice)

    Date = endDate
    if action == 'buy':
        newBal = newBal - amount  
        shareQuantity = amount / sharePrice
    elif action == 'sell':
        newBal = newBal + amount 
        shareQuantity = amount / sharePrice *-1

    trade = {'Date': Date,
        'Buy/Sell': action,
        'Symbol': ticker,
        'Price': sharePrice,
        'Shares': shareQuantity,
        'Amount': amount,
        'cashBalance': newBal}

    stockLedger = stockLedger.append(trade, ignore_index = True)
    return stockLedger

#Place trades here:
loopsDF = statsDFThree['Symbol']
loopsDF = loopsDF.reset_index()
loops = len(loopsDF)

for y in range(loops): 
    try:
        #Pick a ticker
        tradeTicker = statsDFThree['Symbol'].iloc[y]
        buyIndicator = statsDFThree['Buy?'].iloc[y]
        sellIndicator = statsDFThree['Sell?'].iloc[y]
        if buyIndicator == 'yes':
            placeTrade('buy', buyAmt, tradeTicker) 
        elif sellIndicator == 'yes':
            placeTrade('sell', sellAmt, tradeTicker) 
        else:
            continue
    except:
        continue
        

#Save Stock Ledger
stockLedger.to_excel(excelOutputFolder+"Stock Ledger.xlsx")  

#Compile Current Holdings
#aggHoldings = pd.read_excel(excelOutputFolder+"Current Holdings.xlsx", index_col=0)
aggHoldings = stockLedger.groupby(['Symbol']).agg({'Shares':sum,'Date':max})
aggHoldings = aggHoldings.reset_index()
aggHoldings = aggHoldings.set_index('Symbol').join(statsDFTwo.set_index('Symbol'))
aggHoldings = aggHoldings.reset_index()
aggHoldings = aggHoldings[['Symbol','Shares','Date','LastPrice']]
aggHoldings['value'] = aggHoldings['Shares'] * aggHoldings['LastPrice']
#Add Cash Reccord
cashReccord = {'Symbol': 'Cash',
    'Shares': newBal,
    'Date': endDate,
    'LastPrice': 1,
    'value': newBal}
aggHoldings = aggHoldings.append(cashReccord, ignore_index = True)
aggHoldings.to_excel(excelOutputFolder+"Current Holdings.xlsx")  

C:\Users\adamd\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'BRK.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
C:\Users\adamd\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'BF.B', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [2]:
statsStatsDF

,LastPrice,StDev,Avg,Slope,Std/Avg,#ofPeaks,#ofTroughs
count,503.000000,503.000000,503.000000,503.000000,503.000000,503.000000,503.000000
mean,210.995288,12.927469,200.017955,0.162957,0.062740,21.924453,21.385686
std,390.007814,22.934215,367.584726,0.517670,0.031526,5.128844,5.607902
min,11.960000,0.394670,11.869558,-0.804401,0.017464,3.000000,0.000000
25%,61.415001,2.955593,60.896096,-0.025817,0.042409,18.000000,18.000000
50%,117.220001,6.190777,116.208746,0.038397,0.056856,22.000000,22.000000
75%,227.049995,12.498425,214.662223,0.175526,0.074866,25.000000,25.000000
max,5216.009766,192.458137,4993.671848,4.724686,0.262600,40.000000,37.000000


In [59]:
statsDFTwo

,Symbol,LastPrice,Days,StDev,Avg,Slope,Std/Avg,#ofPeaks,#ofTroughs,buyPrice,sellPrice,Buy?,Sell?,Buy or Sell?,Qualifying Stock?
0,MMM,179.660004,126,8.643379,188.716797,-0.203930,0.045801,26.0,30.0,180.073418,197.360176,yes,no,yes,no
1,ABT,125.260002,126,6.409943,120.409023,0.131727,0.053235,25.0,19.0,113.999080,126.818966,no,no,no,no
2,ABBV,115.650002,126,3.989323,112.243855,-0.005214,0.035542,28.0,25.0,108.254532,116.233178,no,no,no,no
3,ABMD,335.320007,126,22.615179,333.433173,0.468687,0.067825,21.0,16.0,310.817994,356.048352,no,no,no,no
4,ACN,363.299988,126,26.530896,324.114529,0.686158,0.081857,22.0,25.0,297.583633,350.645425,no,yes,yes,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,126.959999,125,5.964489,124.400716,0.081006,0.047946,26.0,32.0,118.436227,130.365205,no,no,no,no
499,ZBRA,596.750000,125,32.638439,544.538800,0.436751,0.059938,31.0,32.0,511.900361,577.177239,no,yes,yes,no
500,ZBH,128.809998,125,8.965236,150.711551,-0.209767,0.059486,24.0,14.0,141.746315,159.676787,yes,no,yes,no
501,ZION,65.410004,125,5.044185,57.691040,0.111766,0.087434,33.0,22.0,52.646855,62.735225,no,yes,yes,no


In [4]:
statsDFThree

,Symbol,LastPrice,Days,StDev,Avg,Slope,Std/Avg,#ofPeaks,#ofTroughs,buyPrice,sellPrice,Buy?,Sell?,Buy or Sell?,Qualifying Stock?
94,CNC,73.010002,125,4.723235,68.656400,-0.036224,0.068795,28.0,33.0,63.933165,73.379635,no,no,no,yes
111,CFG,49.610001,125,2.680100,45.594300,0.023994,0.058781,24.0,27.0,42.914200,48.274401,no,yes,yes,yes
146,DFS,114.379997,125,5.242197,123.017470,0.004590,0.042613,22.0,26.0,117.775273,128.259666,yes,no,yes,yes
185,XOM,60.669998,125,3.640733,58.721180,0.030214,0.062000,25.0,27.0,55.080446,62.361913,no,no,no,yes
219,HAL,22.040001,125,2.154797,22.229189,0.013847,0.096935,24.0,26.0,20.074392,24.383987,no,no,no,yes
220,HBI,17.070000,125,0.963352,18.178492,-0.013186,0.052994,22.0,24.0,17.215140,19.141843,yes,no,yes,yes
227,HES,76.070000,125,7.873747,79.548281,-0.021320,0.098981,30.0,28.0,71.674535,87.422028,no,no,no,yes
237,HUM,434.869995,125,22.965616,429.385161,-0.011135,0.053485,30.0,25.0,406.419545,452.350777,no,no,no,yes
272,KIM,23.870001,125,0.994955,21.707946,0.021193,0.045834,22.0,22.0,20.712991,22.702901,no,yes,yes,yes
273,KMI,16.160000,125,0.786220,17.027612,-0.010387,0.046173,22.0,26.0,16.241393,17.813832,yes,no,yes,yes


In [106]:
stockLedger

,Date,Buy/Sell,Symbol,Price,Shares,Amount,cashBalance
0,2021-11-22,buy,SLB,30,3.333333,100,9900
1,2021-11-22,buy,SLB,30,3.333333,100,9800
2,2021-11-22,buy,AAPL,160,0.625000,100,9700


In [110]:
aggHoldings

,Symbol,Shares,Date,LastPrice,value
0,AAPL,1.250000,2021-11-22,160.550003,200.687504
1,MSFT,0.583090,2021-11-22,343.109985,200.064131
2,SLB,6.666667,2021-11-22,30.049999,200.333328
3,Cash,9400.000000,2021-11-22,1.000000,9400.000000
